In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
from train import Trainer

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [4]:
# get data

# note: autougment doesn't seem to work very well 

from models_methods.data.preprocessing import Autoaugment_preprocess
transform = Autoaugment_preprocess(channels=3, resize_dim=(32,32), crop_dim=(28,28)) 


root = "/home/peppe/01_Study/01_University/Semester/2/Intro_to_ML/Project/data" # change this to your data directory

trainset = torchvision.datasets.CIFAR10(root=root, train=True,
                                      download=True, transform=transform.transform)

trainset, valset = torch.utils.data.random_split(trainset, [45000, 5000])

testset = torchvision.datasets.CIFAR10(root=root, train=False,
                                       download=True, transform=transform.transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=64,
                                        shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

data_loaders = {
    "train_loader": trainloader,
    "val_loader": valloader,
    "test_loader": testloader
}          

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# create simple model 

import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
from models_methods.methods.SAM.sam import SAM
from torch.optim.lr_scheduler import StepLR
from models_methods.utility.custom_scheduler import StepLR_SAM

model = SimpleCNN()

optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.005)
scheduler = StepLR(optimizer, step_size=1, gamma=0.01)

In [7]:
sam_optimizer = SAM(model.parameters(), 
                torch.optim.SGD, 
                rho=2, 
                adaptive=True, 
                lr=0.1, momentum=0.9, weight_decay=0.005)
SAM_scheduler = StepLR_SAM(sam_optimizer, step_size=1, gamma=0.01)

In [8]:
simple_net = Trainer(data_loaders=data_loaders,
                     dataset_name="CIFAR10",
                     model=model,
                     optimizer=optimizer,
                     loss_fn=nn.CrossEntropyLoss(label_smoothing=0.1),
                     device=device,
                     seed=42,
                     exp_path="/home/peppe/01_Study/01_University/Semester/2/Intro_to_ML/Project/Code/experiments",
                     exp_name="test_2",
                     use_early_stopping=True,
                     scheduler=scheduler
                    )

In [ ]:
simple_net.main(epochs=2, 
                log_interval=0.1)

In [ ]:
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

preds = simple_net.get_predictions(test=True)
y_true = preds["y_true"]
y_pred = preds["y_pred"]

classes = np.unique(y_true)

conf_matrix = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
for _ in range(2):
    simple_net.main(epochs=2, 
                    log_interval=0.1)

In [14]:
!kill 55682

In [14]:
simple_net.test_step(train=True, recall=True, precision=True)

(0.030842013343175254, 31.92888888888889, 31.20393412484834, 31.92888888888889)

In [ ]:
# visualize with tensorboard
%reload_ext tensorboard
%tensorboard --logdir=/home/peppe/01_Study/01_University/Semester/2/Intro_to_ML/Project/Code/experiments # experiment path

------

# PIM

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn

In [3]:
import os
os.chdir("/home/peppe/01_Study/01_University/Semester/2/Intro_to_ML/Project/Code")

In [4]:
from models_methods.methods.SAM.sam import SAM
from models_methods.methods.PIM.models import builder
from train import Trainer

In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [9]:
# get data

# note: autougment doesn't seem to work very well 

from models_methods.data.preprocessing import Autoaugment_preprocess
transform = Autoaugment_preprocess(channels=3, resize_dim=(520,520), crop_dim=(448,448)) 


root = "/home/peppe/01_Study/01_University/Semester/2/Intro_to_ML/Project/data" # change this to your data directory

trainset = torchvision.datasets.CIFAR10(root=root, train=True,
                                      download=True, transform=transform.transform)

trainset, valset = torch.utils.data.random_split(trainset, [45000, 5000])

testset = torchvision.datasets.CIFAR10(root=root, train=False,
                                       download=True, transform=transform.transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=64,
                                        shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

data_loaders = {
    "train_loader": trainloader,
    "val_loader": valloader,
    "test_loader": testloader
}          

Files already downloaded and verified
Files already downloaded and verified


In [7]:
model = builder.build_efficientnet(num_classes=10)

/home/peppe/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/peppe/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B5_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B5_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
model(trainset[0][0].unsqueeze(0)).keys()

dict_keys(['layer1', 'layer2', 'layer3', 'layer4', 'select_layer1', 'drop_layer1', 'select_layer2', 'drop_layer2', 'select_layer3', 'drop_layer3', 'select_layer4', 'drop_layer4', 'comb_outs'])

In [15]:
pim_simple_net = Trainer(data_loaders=data_loaders,
                         dataset_name="CIFAR10",
                         model=model,
                         optimizer=torch.optim.SGD,
                         loss_fn=nn.CrossEntropyLoss(),
                         device=device,
                         seed=42,
                         exp_path="/home/peppe/01_Study/01_University/Semester/2/Intro_to_ML/Project/Code/experiments",
                         exp_name="test_8",
                         use_SAM=False
                        )

In [ ]:
pim_simple_net.main(epochs=3, 
                    log_interval=5)